## Importing libraries/packages

In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install nltk
# !pip install gensim
# !pip install spacy
# !pip install pyLDAvis
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

In [1]:
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora
import gensim
from gensim.models.coherencemodel import CoherenceModel
import re
# from tabulate import tabulate
import os
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import json
# import image module
from IPython.display import Image


In [2]:
# List stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
print(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Package omw-1.4 is already up-to-date!


## Reading in the data


In [3]:
all_df = pd.DataFrame()
for i in os.listdir('./Climate-change-data_uncompressed'):
    print(i)
    read_df = pd.read_json(f'./Climate-change-data_uncompressed/{i}', lines=True)

    all_df = pd.concat([all_df, read_df])

part-000000000000.json
part-000000000001.json
part-000000000002.json
part-000000000003.json
part-000000000004.json
part-000000000005.json
part-000000000006.json
part-000000000007.json
part-000000000008.json
part-000000000009.json
part-000000000010.json
part-000000000011.json
part-000000000012.json
part-000000000013.json
part-000000000014.json
part-000000000015.json
part-000000000016.json
part-000000000017.json
part-000000000018.json
part-000000000019.json
part-000000000020.json
part-000000000021.json
part-000000000022.json
part-000000000023.json
part-000000000024.json
part-000000000025.json
part-000000000026.json
part-000000000027.json
part-000000000028.json
part-000000000029.json
part-000000000030.json
part-000000000031.json
part-000000000032.json
part-000000000033.json
part-000000000034.json
part-000000000035.json
part-000000000036.json
part-000000000037.json
part-000000000038.json
part-000000000039.json
part-000000000040.json
part-000000000041.json
part-000000000042.json
part-000000

In [4]:
all_df

,copyright,subject_codes,art,modification_datetime,body,company_codes_occur_ticker_exchange,company_codes_occur,company_codes_about,company_codes_lineage,company_codes_ticker_exchange,...,an,word_count,company_codes,industry_codes,title,publication_datetime,publisher_name,action,document_type,dateline
0,"(c) 2009 Dow Jones & Company, Inc.",",gclimt,genv,gsci,gwea,nlet,nlte,gcat,gglobe,n...",,1275316377000,Lomborg may believe global warming is a seriou...,,,,,,...,B000000020091010e5ac0000a,259,,,Barron's Mailbag: Blowing Hot and Cold,1255305600000,"Dow Jones & Company, Inc.",rep,article,NaN
1,"(c) 2004 Dow Jones & Company, Inc.",",c13,c314,gcat,genv,gverg,mcrntg,nedi,opec,c31...",,1084574043000,If there will ever be a time when the U.S. sho...,,,,,,...,B000000020040501e0530000h,1464,,",i16101,i13,i15,i1610109,i1,i16,",Unleash the Potential: The benefits of buildin...,1083542400000,Dow Jones & Company,rep,article,NaN
2,"\n(Copyright (c) 1993, Dow Jones & Co., Inc.)\n",",gcat,genv,",NaN,1034986381000,\nWulff made two further errors: We are not be...,,,,,,...,b000000020011031dpbt001fx,195,,,\nBarron's Mailbag:\nMore On Global Warming\n,754531200000,Dow Jones & Company,rep,article,NaN
3,"Copyright 2019 Dow Jones & Company, Inc. All R...",",gclimt,genv,gcat,gglobe,",,1551138177000,According to the U.S. National Climate Assessm...,,",switcr,ipocch,",,,,...,B000000020190216ef2i0015v,911,",aupra,bnkeng,bp,ipocch,switcr,switcr,",,Climate Change's Risk to Investors --- Why spe...,1550448000000,"Dow Jones & Company, Inc.",rep,article,NaN
4,"Copyright 2022 Dow Jones & Company, Inc. All R...",",gpir,gpol,gvote,ncolu,nlet,gcat,ncat,nfact,nf...",,1667611872000,To the Editor: Both parties need to realize th...,",GNGYF:PSGM,GRCH:XFRA,601333:XSHG,0525:XHKG,GO...",",onlnfr,imonf,guarly,goog,bisett,applc,amzcom,",",imonf,",,",GOOG:NEOE,GOOG:XLIM,GOOC:XWBO,0RIH:XLON,GOOGL...",...,B000000020221104eib7000p1,951,",amzcom,amzcom,applc,applc,bisett,bisett,goog,...",",i1,",Fossil Fuels Are the Critical Issue,1667779200000,"Dow Jones & Company, Inc.",rep,article,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31340,Copyright © 2021 West Elgin Chronicle,",gclimt,nlet,gcat,genv,gglobe,ncat,nfact,nfce,...",,1638422916000,Set aside a corner in the living room for Sant...,,,,,,...,WSTELGIN20211202ehc20000h,158,,,Why are we buying Christmas trees?,1638403200000,Postmedia Network Inc.,add,article,London
31341,"Copyright 2022, Hagadone Corporation. All Righ...",",gnatcn,gdroug,gnrsca,gclimt,nlet,gcat,gdis,ge...",,1646794734000,Drought conditions in most of Montana are curr...,,,,,,...,WSTNWSMT20220309ei3800007,333,,,Megadrought highlights need for water conserva...,1646697600000,Hagadone Corp.,add,article,NaN
31342,"Copyright 2016, The Weatherford Democrat. All ...",",gclimt,gcat,genv,gglobe,",,1461943637000,They have found that reforestation also plays ...,,,,,,...,WTHFRDDM20160429ec4s00001,473,,,Researcher finds that New Hampshire's growing ...,1461801600000,"Community Newspaper Holdings, Inc.",add,article,NaN
31343,"Copyright 2017, Waterloo-Cedar Falls Courier. ...",",gvlwh,gclimt,gcat,genv,gglobe,gpir,gpol,gvbod...",,1494368093000,The two-term incumbent faced regular interrupt...,,,,,,...,WTRLCDRF20170509ed590003s,826,",epagy,uscbof,ushor,",,Dubuque town hall puts heat on Blum,1494288000000,Lee Enterprises,add,article,NaN


## Preprocessing the dataset & sampling

In [5]:
# Run this cell if you find any column names that don't make sense to be used.
# Replace the words 'index' and 'Unnamed: 0' with the names that you want to exclude.

# Drop the column name making less sense  , 'subject_codes'
all_df = all_df.reset_index().drop(columns=['copyright','company_codes_association', 'art', 'modification_datetime', 'company_codes_occur_ticker_exchange', 'company_codes_occur', 'company_codes_about', 'company_codes_lineage','company_codes_ticker_exchange','company_codes_relevance_ticker_exchange','publication_date',	'market_index_codes','credit','section','company_codes_association_ticker_exchange','currency_codes','an','word_count','company_codes','industry_codes','action','document_type','dateline', 'publication_datetime','company_codes_relevance','source_code','person_codes','company_codes_lineage_ticker_exchange','ingestion_datetime','modification_date','company_codes_about_ticker_exchange'])

In [6]:
all_df

,index,subject_codes,body,snippet,region_of_origin,source_name,language_code,region_codes,byline,title,publisher_name
0,0,",gclimt,genv,gsci,gwea,nlet,nlte,gcat,gglobe,n...",Lomborg may believe global warming is a seriou...,"To the Editor:\n\nThe Sept. 28 article ""A Smar...",NAMZ USA,Barron's,en,",usa,namz,",,Barron's Mailbag: Blowing Hot and Cold,"Dow Jones & Company, Inc."
1,1,",c13,c314,gcat,genv,gverg,mcrntg,nedi,opec,c31...",If there will ever be a time when the U.S. sho...,Crude-oil prices are at $35 a barrel and OPEC ...,NAMZ USA,Barron's,en,",usa,usca,use,usfl,usny,uss,ustx,usw,namz,",By John Sillin,Unleash the Potential: The benefits of buildin...,Dow Jones & Company
2,2,",gcat,genv,",\nWulff made two further errors: We are not be...,"\nTo the Editor:\n\n\n\nBarry Wulff (Mailbag, ...",NAMZ USA,Barron's,en,",namz,usa,",NaN,\nBarron's Mailbag:\nMore On Global Warming\n,Dow Jones & Company
3,3,",gclimt,genv,gcat,gglobe,",According to the U.S. National Climate Assessm...,Radical changes in energy generation and consu...,NAMZ USA,Barron's,en,,By Matthew C. Klein,Climate Change's Risk to Investors --- Why spe...,"Dow Jones & Company, Inc."
4,4,",gpir,gpol,gvote,ncolu,nlet,gcat,ncat,nfact,nf...",To the Editor: Both parties need to realize th...,To the Editor: I think there is only one criti...,NAMZ USA,Barron's,en,",usa,namz,",,Fossil Fuels Are the Critical Issue,"Dow Jones & Company, Inc."
...,...,...,...,...,...,...,...,...,...,...,...
2548102,31340,",gclimt,nlet,gcat,genv,gglobe,ncat,nfact,nfce,...",Set aside a corner in the living room for Sant...,I watched the news and the devastation in Brit...,CANA NAMZ,West Elgin Chronicle,en,",cabc,cana,namz,",Cameron Bain,Why are we buying Christmas trees?,Postmedia Network Inc.
2548103,31341,",gnatcn,gdroug,gnrsca,gclimt,nlet,gcat,gdis,ge...",Drought conditions in most of Montana are curr...,To the editor:\n\nThe American West is in a me...,NAMZ USA,The Western News,en,",usa,namz,",,Megadrought highlights need for water conserva...,Hagadone Corp.
2548104,31342,",gclimt,gcat,genv,gglobe,",They have found that reforestation also plays ...,"By meteorological standards, weather condition...",NAMZ USA,The Weatherford Democrat,en,",ustx,namz,usa,uss,",Breanna Edelstein | Community News Content,Researcher finds that New Hampshire's growing ...,"Community Newspaper Holdings, Inc."
2548105,31343,",gvlwh,gclimt,gcat,genv,gglobe,gpir,gpol,gvbod...",The two-term incumbent faced regular interrupt...,"DUBUQUE — U.S. Rep. Rod Blum, R-Iowa, met a ho...",NAMZ USA,Waterloo-Cedar Falls Courier,en,",usa,usia,namz,usc,",THOMAS J. BARTON thomas.barton@thmedia.com,Dubuque town hall puts heat on Blum,Lee Enterprises


In [7]:
sample_df=all_df.sample(600)

In [8]:
sample_df

,index,subject_codes,body,snippet,region_of_origin,source_name,language_code,region_codes,byline,title,publisher_name
703875,11427,",gclimt,genv,gpol,guni,nedc,gcat,gedu,gglobe,g...","CORVALLIS, Ore. -- With all the evidence cropp...",U-WIRE-03/05/2004-Oregon State U.: COLUMN: Stu...,NAMZ USA,U-Wire,en,",usa,usor,namz,usw,",,Oregon State U.: COLUMN: Studying the science ...,"M2 Communications, Ltd."
1221560,25372,",gclimt,gcycl,gcat,genv,gglobe,gspo,ncat,nfact...",The cyclist educated and encouraged people en-...,Cyclist Abid Baig has reportedly undertaken th...,ASIA PAKIS SASIAZ,The Balochistan Times,en,",pakis,asiaz,devgcoz,dvpcoz,indsubz,sasiaz,",,Longest and fastest solo bike ride: Abid pedal...,Daily Balochistan Times'
399560,21768,",gqual,gairp,gclimt,gcat,genv,gglobe,",Our news journalists obtained a quote from the...,2019 JUL 29 (VerticalNews) -- By a News Report...,NAMZ USA,Global Warming Focus,en,",porl,turk,asiaz,balkz,devgcoz,dvpcoz,eecz,eur...",,Environment - Atmospheric Pollution; Findings ...,"NewsRX, LLC"
689773,28840,",gcivds,gclimt,gcrim,gcat,gcns,genv,gglobe,gri...","The conference took a day off yesterday, thoug...",COPENHAGEN — Danish police released hundreds o...,ASIA SEASIAZ SINGP,TODAY (Singapore),en,",den,eecz,eurz,nordz,scandz,weurz,",,"Nearly 1,000 climate protesters released",MediaCorp Press Ltd
1186743,22118,",gclimt,gcat,genv,gglobe,",Our news editors obtained a quote from the res...,2020 APR 13 (VerticalNews) -- By a News Report...,NAMZ USA,Global Warming Focus,en,",china,chinaz,beijin,apacz,asiaz,bric,devgcoz,...",,Climate Modeling; Chinese Academy of Sciences ...,"NewsRX, LLC"
...,...,...,...,...,...,...,...,...,...,...,...
386950,9158,",gsust,gclimt,npag,gcat,genv,gglobe,ncat,",MONTEREY >> Monterey is expected to soon exper...,By Dennis L. Taylor\n\ndtaylor@montereyherald.com,NAMZ USA,Monterey County Herald,en,",usa,namz,",,Environment; Monterey will see more required w...,MediaNews Group Inc.
767472,12403,",gclimt,gcat,genv,gglobe,gwea,",Mwenda was addressing African journalists who ...,"Nov 26, 2012 (New Vision/All Africa Global Med...",AFRICA SAFR SOUAFR,All Africa,en,",qatar,africaz,doha,jap,kenya,usa,apacz,asiaz,...",by Catherine Mwesigwa Kizza,Africa Wants World Carbon Emissions Cut,All Africa Global Media
2138652,31281,",gmovie,gclimt,gfesti,gflood,nnam,gcat,gdis,ge...",The film was directed by Fisher Stevens. It ch...,DiCaprio unveils climate change film 'Before t...,NAMZ USA,Telegraph Herald,en,",usa,namz,",The Associated Press,DiCaprio unveils climate change film 'Before t...,Woodward Communications Inc.
1417980,1794,",gclimt,gcat,genv,gglobe,gwea,","Previously, officials believed the park 60 mil...","ROCKY MOUNTAIN NATIONAL PARK, Colo.- Geologist...",NAMZ USA,Houston Chronicle,en,",usa,namz,",JOSEPH B. VERRENGIA; Associated Press,Glacier growth surprises geologists Despite gl...,"Hearst Communications, Inc."


In [9]:
# pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')


def preprocess_data(df):
    # Here the article's content is in the column named 'content'
    # If it resides under other column name, then change the word content in the next line with
    # the name of your column
    # Remove punctuation characters
    #df['preprocessed_sentence'] = sample_df.body.apply(lambda x: re.sub(r"[^a-zA-Z0-9]", " ", str(x)))
    df['preprocessed_sentence'] = sample_df['body']
    # Removing numbers
    #df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r"[\d+$]", " ", str(x)))
    # df['preprocessed_sentence']=df['body']
    #Removing new lines and tabs
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r"\n", " ", str(x)))
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r"\t", " ", str(x)))

    # Remove stopwords, lower case all the words and split them into a list of words
    # df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: pattern.sub('', re.sub(r"[^a-zA-Z0-9]", " ", x).lower()))

    df['processed_word_count'] = df.preprocessed_sentence.apply(lambda x: len(x))
    df = df[df.processed_word_count > 0]

    return df


In [10]:
temp_df = preprocess_data(sample_df)

In [11]:
temp_df

,index,subject_codes,body,snippet,region_of_origin,source_name,language_code,region_codes,byline,title,publisher_name,preprocessed_sentence,processed_word_count
703875,11427,",gclimt,genv,gpol,guni,nedc,gcat,gedu,gglobe,g...","CORVALLIS, Ore. -- With all the evidence cropp...",U-WIRE-03/05/2004-Oregon State U.: COLUMN: Stu...,NAMZ USA,U-Wire,en,",usa,usor,namz,usw,",,Oregon State U.: COLUMN: Studying the science ...,"M2 Communications, Ltd.","CORVALLIS, Ore. -- With all the evidence cropp...",5419
1221560,25372,",gclimt,gcycl,gcat,genv,gglobe,gspo,ncat,nfact...",The cyclist educated and encouraged people en-...,Cyclist Abid Baig has reportedly undertaken th...,ASIA PAKIS SASIAZ,The Balochistan Times,en,",pakis,asiaz,devgcoz,dvpcoz,indsubz,sasiaz,",,Longest and fastest solo bike ride: Abid pedal...,Daily Balochistan Times',The cyclist educated and encouraged people en-...,316
399560,21768,",gqual,gairp,gclimt,gcat,genv,gglobe,",Our news journalists obtained a quote from the...,2019 JUL 29 (VerticalNews) -- By a News Report...,NAMZ USA,Global Warming Focus,en,",porl,turk,asiaz,balkz,devgcoz,dvpcoz,eecz,eur...",,Environment - Atmospheric Pollution; Findings ...,"NewsRX, LLC",Our news journalists obtained a quote from the...,2247
689773,28840,",gcivds,gclimt,gcrim,gcat,gcns,genv,gglobe,gri...","The conference took a day off yesterday, thoug...",COPENHAGEN — Danish police released hundreds o...,ASIA SEASIAZ SINGP,TODAY (Singapore),en,",den,eecz,eurz,nordz,scandz,weurz,",,"Nearly 1,000 climate protesters released",MediaCorp Press Ltd,"The conference took a day off yesterday, thoug...",949
1186743,22118,",gclimt,gcat,genv,gglobe,",Our news editors obtained a quote from the res...,2020 APR 13 (VerticalNews) -- By a News Report...,NAMZ USA,Global Warming Focus,en,",china,chinaz,beijin,apacz,asiaz,bric,devgcoz,...",,Climate Modeling; Chinese Academy of Sciences ...,"NewsRX, LLC",Our news editors obtained a quote from the res...,2799
...,...,...,...,...,...,...,...,...,...,...,...,...,...
386950,9158,",gsust,gclimt,npag,gcat,genv,gglobe,ncat,",MONTEREY >> Monterey is expected to soon exper...,By Dennis L. Taylor\n\ndtaylor@montereyherald.com,NAMZ USA,Monterey County Herald,en,",usa,namz,",,Environment; Monterey will see more required w...,MediaNews Group Inc.,MONTEREY >> Monterey is expected to soon exper...,4053
767472,12403,",gclimt,gcat,genv,gglobe,gwea,",Mwenda was addressing African journalists who ...,"Nov 26, 2012 (New Vision/All Africa Global Med...",AFRICA SAFR SOUAFR,All Africa,en,",qatar,africaz,doha,jap,kenya,usa,apacz,asiaz,...",by Catherine Mwesigwa Kizza,Africa Wants World Carbon Emissions Cut,All Africa Global Media,Mwenda was addressing African journalists who ...,6667
2138652,31281,",gmovie,gclimt,gfesti,gflood,nnam,gcat,gdis,ge...",The film was directed by Fisher Stevens. It ch...,DiCaprio unveils climate change film 'Before t...,NAMZ USA,Telegraph Herald,en,",usa,namz,",The Associated Press,DiCaprio unveils climate change film 'Before t...,Woodward Communications Inc.,The film was directed by Fisher Stevens. It ch...,1515
1417980,1794,",gclimt,gcat,genv,gglobe,gwea,","Previously, officials believed the park 60 mil...","ROCKY MOUNTAIN NATIONAL PARK, Colo.- Geologist...",NAMZ USA,Houston Chronicle,en,",usa,namz,",JOSEPH B. VERRENGIA; Associated Press,Glacier growth surprises geologists Despite gl...,"Hearst Communications, Inc.","Previously, officials believed the park 60 mil...",3156


In [ ]:
temp_df[['preprocessed_sentence', 'snippet','title','subject_codes']].to_csv('climate-change-sample.csv')
